# Import Libraries

In [1]:
import pandas as pd
import numpy as np 
import os
import sys
import logging

In [2]:
module_path =os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\scripts")

if module_path not in sys.path:
    sys.path.append(module_path+"\\model")



logging.basicConfig(filename='../logs/notebook.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',level=logging.DEBUG)


In [3]:
#import our self defined scripts
from dvc_data import DVCDATA

In [23]:
instance=DVCDATA()
traindata,dataurl,version=instance.get_data('pharmtec train','data/train.csv','https://github.com/Blvisse/Pharmtec_Sales',"'train-v1'")


Fetching data from Repo ... 

Converting into dataframe 

Logging data deatails to mlflow 

Done....


In [24]:
traindata

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [26]:
instance=DVCDATA()
testdata,dataurl,version=instance.get_data('pharmtec test','data/test.csv','https://github.com/Blvisse/Pharmtec_Sales',"'test-v1'")

Fetching data from Repo ... 

Converting into dataframe 

Logging data deatails to mlflow 

Done....


In [27]:
testdata

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0
41084,41085,1112,6,2015-08-01,1.0,0,0,0
41085,41086,1113,6,2015-08-01,1.0,0,0,0
41086,41087,1114,6,2015-08-01,1.0,0,0,0


In [29]:
data,dataurl,version=instance.get_data('pharmtec store','data/store.csv','https://github.com/Blvisse/Pharmtec_Sales',"'store-v1'")

Fetching data from Repo ... 

Converting into dataframe 

Logging data deatails to mlflow 

Done....


# EDA

Now that we got the data we create we need to do some analysis on the data

## Getting Duplicated values

In [15]:
from clean_data import CLEAN

In [17]:
cleanInstance=CLEAN(traindata)
cleanInstance.calculate_null()

The report of missing values is as follows


,Total Missing,Percentage Missing
Store,0.0,0.0
DayOfWeek,0.0,0.0
Date,0.0,0.0
Sales,0.0,0.0
Customers,0.0,0.0
Open,0.0,0.0
Promo,0.0,0.0
StateHoliday,0.0,0.0
SchoolHoliday,0.0,0.0


There seems to be no null values in the training dataset

In [18]:
cleanInstance=CLEAN(testdata)
cleanInstance.calculate_null()

The report of missing values is as follows


,Total Missing,Percentage Missing
Id,0.0,0.000000
Store,0.0,0.000000
DayOfWeek,0.0,0.000000
Date,0.0,0.000000
Open,11.0,0.026772
Promo,0.0,0.000000
StateHoliday,0.0,0.000000
SchoolHoliday,0.0,0.000000


We have a total of 11 missing values in the test data with all of them being in the open column 

In [19]:
cleanInstance=CLEAN(storedata)
cleanInstance.calculate_null()

The report of missing values is as follows


,Total Missing,Percentage Missing
Store,0.0,0.000000
StoreType,0.0,0.000000
Assortment,0.0,0.000000
CompetitionDistance,3.0,0.269058
CompetitionOpenSinceMonth,354.0,31.748879
CompetitionOpenSinceYear,354.0,31.748879
Promo2,0.0,0.000000
Promo2SinceWeek,544.0,48.789238
Promo2SinceYear,544.0,48.789238
PromoInterval,544.0,48.789238


We do have a couple of missing values in this row. We need to address them 

## Missing Data

In [30]:
# we have a look at the mssing data in the columns

### Competition Distance

In [33]:
storedata[storedata['CompetitionDistance'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


We see that the 3 throws that don't have details on competition distance also don't have details on competionOpenSinceMonth and CompetitionOpenSinceYear. We can fill in the missing values with the median value of the column instead of filling with zero which will be interprated as the competition is next to the store 

In [35]:
storedata['CompetitionDistance'].fillna(storedata['CompetitionDistance'].median(),inplace=True)

In [37]:
#check to see if they were resolved
storedata[storedata['CompetitionDistance'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


### Copetition Open Since Year

In [34]:
storedata[storedata['CompetitionOpenSinceYear'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
11,12,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
12,13,d,a,310.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
15,16,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN
18,19,a,c,3240.0,NaN,NaN,1,22.0,2011.0,"Mar,Jun,Sept,Dec"
21,22,a,a,1040.0,NaN,NaN,1,22.0,2012.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
1095,1096,a,c,1130.0,NaN,NaN,1,10.0,2014.0,"Mar,Jun,Sept,Dec"
1099,1100,a,a,540.0,NaN,NaN,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


Looking at the value we do see the competition distance is available and hence there exists a competitior and hence need to fill in the opensincemonth column we shall also impute this with median of the column and do the same for the year as well

In [38]:
storedata['CompetitionOpenSinceYear'].fillna(storedata['CompetitionOpenSinceYear'].median(),inplace=True)
storedata['CompetitionOpenSinceMonth'].fillna(storedata['CompetitionOpenSinceMonth'].median(),inplace=True)

In [39]:
#check to see if they were resolved
storedata[storedata['CompetitionOpenSinceMonth'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


### Promo 2 Since Week

In [40]:
storedata[storedata['Promo2SinceWeek'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1107,1108,a,a,540.0,4.0,2004.0,0,NaN,NaN,NaN
1109,1110,c,c,900.0,9.0,2010.0,0,NaN,NaN,NaN
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,8.0,2010.0,0,NaN,NaN,NaN


Logically speaking if promo2 doesn't exist there shouldn't be a start or end week to it. So we begin by filling rows whih had promo2 as 0 we impute with 0 as well 

In [42]:
storedata['Promo2SinceWeek']=np.where(storedata['Promo2'] == 0, 0,storedata['Promo2SinceWeek'])

In [43]:
storedata[storedata['Promo2SinceWeek'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


Seems to have solved our problem...we also apply it to the sinceyear column

In [44]:
storedata['Promo2SinceYear']=np.where(storedata['Promo2'] == 0, 0,storedata['Promo2SinceYear'])

In [45]:
storedata[storedata['Promo2SinceYear'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [46]:
storedata[storedata['PromoInterval'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,NaN
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,NaN
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,NaN
5,6,a,a,310.0,12.0,2013.0,0,0.0,0.0,NaN
6,7,a,c,24000.0,4.0,2013.0,0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1107,1108,a,a,540.0,4.0,2004.0,0,0.0,0.0,NaN
1109,1110,c,c,900.0,9.0,2010.0,0,0.0,0.0,NaN
1111,1112,c,c,1880.0,4.0,2006.0,0,0.0,0.0,NaN
1112,1113,a,c,9260.0,8.0,2010.0,0,0.0,0.0,NaN


In [47]:
storedata['PromoInterval']=np.where(storedata['Promo2'] == 0, 0,storedata['PromoInterval'])

In [48]:
storedata[storedata['PromoInterval'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [49]:
cleanInstance=CLEAN(storedata)
cleanInstance.calculate_null()

The report of missing values is as follows


,Total Missing,Percentage Missing
Store,0.0,0.0
StoreType,0.0,0.0
Assortment,0.0,0.0
CompetitionDistance,0.0,0.0
CompetitionOpenSinceMonth,0.0,0.0
CompetitionOpenSinceYear,0.0,0.0
Promo2,0.0,0.0
Promo2SinceWeek,0.0,0.0
Promo2SinceYear,0.0,0.0
PromoInterval,0.0,0.0


Done with that so we move on to the next dataset

### Open

In [50]:
testdata[testdata['Open'].isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


Logically speaking you can't run a promo while a store is closed hence we fill in those with 1 indicatinf the store is open

In [52]:
testdata['Open']=np.where(testdata['Promo'] == 1, 1,testdata['Open'])

In [53]:
testdata[testdata['Open'].isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0
10751,10752,622,6,2015-09-05,NaN,0,0,0


For the remaining ones we fill in with zeros indicating that the shop might have been closed

In [58]:
testdata['Open']=np.where(testdata['Open'].isnull() , 0,testdata['Open'])

In [59]:
testdata[testdata['Open'].isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday


We are done with that 

#### Save cleaned data sets

In [60]:
testdata.to_csv('../data/clean_test.csv',index=False)
storedata.to_csv('../data/clean_store.csv',index=False)

We then store this data on our dvc repo where we can access it anytime